In [47]:
import pandas as pd
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner, DirichletPartitioner, PathologicalPartitioner
from flwr_datasets.visualization import plot_label_distributions
from datasets import Dataset

In [55]:
fds_main = FederatedDataset(
    dataset="scikit-learn/adult-census-income",
    partitioners={"train": 3}
)

In [56]:
part0 = fds_main.load_partition(0)
part1 = fds_main.load_partition(1)
part2 = fds_main.load_partition(2)

In [57]:
split_ds_0 = {"train": part0}
split_ds_1 = {"train": part1}
split_ds_2 = {"train": part2}

In [58]:
fds_iid = FederatedDataset(
    dataset=None,
    partitioners={"train": 3},
    preprocessor=lambda _: split_ds_0
)

fds_dir1 = FederatedDataset(
    dataset=None,
    partitioners={"train": DirichletPartitioner(num_partitions=2, partition_by="income", alpha=0.5, min_partition_size=2500)},
    preprocessor=lambda _: split_ds_1
)

fds_dir1 = FederatedDataset(
    dataset=None,
    partitioners={"train": DirichletPartitioner(num_partitions=2, partition_by="workclass", alpha=0.5, min_partition_size=2000)},
    preprocessor=lambda _: split_ds_2
)

c:\Users\prem2\miniconda3\envs\dataAnSc\Lib\site-packages\flwr_datasets\utils.py:109: UserWarning: The currently tested dataset are ['mnist', 'ylecun/mnist', 'cifar10', 'uoft-cs/cifar10', 'fashion_mnist', 'zalando-datasets/fashion_mnist', 'sasha/dog-food', 'zh-plus/tiny-imagenet', 'scikit-learn/adult-census-income', 'cifar100', 'uoft-cs/cifar100', 'svhn', 'ufldl-stanford/svhn', 'sentiment140', 'stanfordnlp/sentiment140', 'speech_commands', 'LIUM/tedlium', 'flwrlabs/femnist', 'flwrlabs/ucf101', 'flwrlabs/ambient-acoustic-context', 'jlh/uci-mushrooms', 'Mike0307/MNIST-M', 'flwrlabs/usps', 'scikit-learn/iris', 'flwrlabs/pacs', 'flwrlabs/cinic10', 'flwrlabs/caltech101', 'flwrlabs/office-home', 'flwrlabs/fed-isic2019']. Given: None.
  warnings.warn(


In [60]:
fds_dir1.load_partition(0)['train']

TypeError: argument should be a str or an os.PathLike object where __fspath__ returns a str, not 'NoneType'

In [63]:
_ = plot_label_distributions(
    partitioner=fds_iid.partitioners["train"],
    label_name="income",
    legend=True,
)

TypeError: argument should be a str or an os.PathLike object where __fspath__ returns a str, not 'NoneType'

In [64]:
from datasets import load_dataset, Dataset
from flwr_datasets.partitioner import IidPartitioner
from flwr_datasets import FederatedDataset

# Step 1: Load the original dataset from Hugging Face
dataset = load_dataset("scikit-learn/adult-census-income", split="train")

# Step 2: Create 3 top-level IID partitions
top_level_partitioner = IidPartitioner(num_partitions=3)

# Partition into 3 parts
top_level_partitions = top_level_partitioner(dataset)

# Step 3: For each top-level partition, further split into 2 sub-partitions
all_sub_partitions = []

for top_i, top_partition in enumerate(top_level_partitions):
    # Wrap the list of examples into a Hugging Face Dataset
    ds_top = Dataset.from_dict(top_partition)
    
    # Sub-partition this chunk into 2 parts
    sub_partitioner = IidPartitioner(num_partitions=2)
    sub_parts = sub_partitioner(ds_top)
    
    for sub_i, sub_partition in enumerate(sub_parts):
        ds_sub = Dataset.from_dict(sub_partition)
        all_sub_partitions.append((f"client_{top_i}_{sub_i}", ds_sub))

# Now you have 6 client partitions
for client_id, ds in all_sub_partitions:
    print(f"{client_id}: {len(ds)} samples")



TypeError: 'IidPartitioner' object is not callable